<a href="https://colab.research.google.com/github/vishalmysore/AI/blob/main/examples/llamaindex/RequirementDocument.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-p

In [2]:
!pip install llama-index chromadb --quiet
!pip install chromadb==0.4.17
!pip install sentence-transformers
!pip install pydantic==1.10.11
!pip install open-clip-torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 4

In [1]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.storage.storage_context import StorageContext
from llama_index.embeddings import HuggingFaceEmbedding
from IPython.display import Markdown, display
import chromadb

In [2]:
import os
import openai
from google.colab import userdata


os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [39]:
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/vishalmysore/cookGPT/main/docs/cookGPT_Tech_Requirement.pdf' -O 'coogGPTTechRequirement.pdf'


--2024-01-28 19:44:42--  https://raw.githubusercontent.com/vishalmysore/cookGPT/main/docs/cookGPT_Tech_Requirement.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39431 (39K) [application/octet-stream]
Saving to: ‘coogGPTTechRequirement.pdf’

coogGPTTechRequirem 100%[===================>]  38.51K  --.-KB/s    in 0.004s  

2024-01-28 19:44:42 (10.5 MB/s) - ‘coogGPTTechRequirement.pdf’ saved [39431/39431]



In [3]:
from llama_index import ServiceContext

# create client and a new collection
chroma_client = chromadb.EphemeralClient()
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")





In [4]:
chroma_collection = chroma_client.get_or_create_collection("chitraGPT")
chroma_client.delete_collection("chitraGPT")
chroma_collection = chroma_client.get_or_create_collection("chitraGPT")
# define embedding function


In [8]:

def store_chroma(dir) :
  # load documents
  documents = SimpleDirectoryReader(dir).load_data()

  # set up ChromaVectorStore and load in data
  vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
  storage_context = StorageContext.from_defaults(vector_store=vector_store)
  service_context = ServiceContext.from_defaults(embed_model=embed_model)
  index = VectorStoreIndex.from_documents(
      documents, storage_context=storage_context, service_context=service_context
  )
  return index

In [18]:
def query(query,index):
  # Query Data
  query_engine = index.as_query_engine()
  response = query_engine.query(query)
  display(Markdown(f"<b>{response}</b>"))
  im = gr.Image(label="Diagram")
  return [im,response]

In [1]:
!pip install gradio

In [4]:
!pip install Werkzeug

In [19]:
import gradio as gr
from werkzeug.utils import secure_filename
import os


def upload_file(file,chormaIndex1):
    directory_part = os.path.dirname(file)
    chormaIndex1=store_chroma(directory_part)
    print("Directory part:", directory_part)

    return {output_col: gr.Column(visible=True),
            file_output: file,
            chormaIndex:chormaIndex1
            }
with gr.Blocks() as demo:
    file_output = gr.File()
    chormaIndex = gr.State([])
    upload_button = gr.UploadButton("Click to Upload a File", file_types=[".pdf"], file_count="single")
    with gr.Column(visible=False) as output_col:
          im = gr.Image(label="Diagram")
          outBox = gr.Textbox(label="Output")
          formated_rec_box = gr.Textbox("get all the components ",label="Type your query here")
          btnSubmit = gr.Button(value="Create Diagram")
          btnSubmit.click(query,inputs=[formated_rec_box,chormaIndex],outputs=[im,outBox])
    upload_button.upload(upload_file,inputs=[upload_button,chormaIndex], outputs=[file_output,output_col,chormaIndex])
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e9e2e4c68920dd2b05.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Directory part: /tmp/gradio/6551841c17d8ae05b59198d7d7b7e12c1255ebdc


<b>The components mentioned in the given context are as follows:
1. Integration
   - External Platforms
     - Requirement
     - Specification
2. Sequence of Events
   - Recipe Generation Flow
     - Requirement
     - Specification
3. Deployment
   - Deployment Environment
     - Requirement
     - Specification</b>

<b>The components of the application include the external platforms and devices, smart kitchen appliances, APIs, voice-activated assistant, user interface, frontend, backend, AI model, and the deployment environment. The interaction between these components involves exploring integration possibilities with external platforms and devices, investigating APIs for potential collaborations with smart kitchen devices, exploring voice-activated assistant integration possibilities, defining the sequence of events for recipe generation, processing user input, validating and formatting input, integrating the AI model, generating a recipe, and presenting the recipe to the user with relevant images and instructions.</b>

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e9e2e4c68920dd2b05.gradio.live
